In [17]:
import json
import pandas as pd    

dreamprm_meta = "../../DreamPRM/data/meta.json"
dreamprm_train = "../../DreamPRM/data/train.json"
MATH_path = "../MATH"

import os
MATH = {'train':{}, 'test':{}}
for i in MATH:
    path = os.path.join(MATH_path, i)
    for j in os.listdir(path):
        MATH[i][j] = [os.path.join(path, j, k) for k in os.listdir(os.path.join(path, j)) if k.endswith('.json')]
        

prm800k_path = "../prm800k/prm800k/data"
prm_split_path = {"test": "../prm800k/prm800k/math_splits/test.jsonl", 
                 "train": "../prm800k/prm800k/math_splits/train.jsonl"}
prm800k  = {}

for phase in range(1,3):
    prm800k[f"phase{phase}"] = {}
    for type in ['train', 'test']:
        prm800k[f"phase{phase}"][type] = f"{prm800k_path}/phase{phase}_{type}.jsonl"
        


In [18]:
ds = prm800k["phase2"]
ds = pd.read_json(path_or_buf=ds["train"], lines=True)

In [19]:
ds['correctness'] = ds['label'].apply(lambda x: x["finish_reason"])

In [20]:
ds["correctness"].unique()
## find each unique count
ds['correctness'].value_counts()

correctness
found_error    82003
solution       11665
give_up         3839
bad_problem      275
Name: count, dtype: int64

In [21]:
ds['label'].iloc[0]['steps'][0]['completions'][0]['rating']

1

In [22]:
from datasets import  Dataset, concatenate_datasets, DatasetDict
import pandas as pd
from tqdm import tqdm


splits = ["train", "test"]


def augment(row):
    new_completions = []
    new_labels = []
    for t, l in zip(row["trajectories"], row["trajectories_labels"]):
        new_completions.append(row["completions"] + [t])
        new_labels.append(row["labels"] + [l])
    row["new_completions"] = new_completions
    row["new_labels"] = new_labels
    return row



def unnest_dataset_lists(
    dataset: Dataset,
    list_columns: str | list[str],
) -> Dataset:
    """
    Unnest list columns in a datasets.Dataset where each row contains a list of lists.
    Each inner list becomes a new row, maintaining alignment across specified columns.
    
    Args:
        dataset: The input datasets.Dataset
        list_columns: Column name(s) containing lists to unnest
        
    Returns:
        A new datasets.Dataset with unnested rows
    """
    df = dataset.to_pandas()
    
    if isinstance(list_columns, str):
        list_columns = [list_columns]
    
    # Create rows for each completion/label pair
    rows = []
    
    for idx, row in df.iterrows():
        # Get number of completions for this row
        num_completions = len(row[list_columns[0]])
        
        # For each completion
        for completion_idx in range(num_completions):
            new_row = {}
            
            # Copy non-list columns
            for col in df.columns:
                if col not in list_columns:
                    new_row[col] = row[col]
            
            # Add index to keep the reference to the prompt
            new_row['index'] = idx
            
            # Add list columns
            for col in list_columns:
                new_row[col] = row[col][completion_idx]
            
            rows.append(new_row)
    
    # Create new dataframe
    result_df = pd.DataFrame(rows)
    
    # Convert back to datasets.Dataset
    return Dataset.from_pandas(result_df)

full_ds = DatasetDict()

for split in splits:
    print("Split:", split)
    datasets = []
    for i, ds in enumerate(prm800k):
        ds = prm800k[ds]
        ds = pd.read_json(path_or_buf=ds[split], lines=True)
        ds['correctness'] = ds['label'].apply(lambda x: x["finish_reason"])
        print("phase:", i + 1)
        new_ds = []
        for row in tqdm(ds.iterrows(), total=len(ds)):
            row = row[1]  # Get the row data
            solution = []
            trajectories = []
            trajectories_labels = []
            label = []
            new_row = {}
            new_row["prompt"] = row["question"]["problem"]

            for step in row["label"]["steps"]:
                completions = step["completions"]
                if not completions:
                    continue
                if step["chosen_completion"] is None:
                    # Haven't found any relevant point here, discard them for the moment
                    #continue
                    if len(completions) > 1:
                        trajectories = []
                        trajectories_labels = []
                        for c in completions:
                            trajectories.append(c["text"])
                            trajectories_labels.append(not (c["rating"] == -1))
                        # We can continue here assuming there shouldn't be any more trajectories after this point
                        continue

                else:
                    completion = completions[step["chosen_completion"]]
                    solution.append(completion["text"])
                    label.append(not (completion["rating"] == -1))
                    continue

            new_row["trajectories"] = trajectories
            new_row["trajectories_labels"] = trajectories_labels
            new_row["completions"] = solution
            new_row["labels"] = label
            new_row['correctness'] = row['correctness']
            new_ds.append(new_row)

        new_ds = Dataset.from_list(new_ds)

        # Augment the dataset by adding the trajectories as completions
        ds_updated = new_ds.map(augment, num_proc=8)

        # Similar to a pd.DataFrame.explode, get a row per each nested list in the dataset,
        # so that we have all the possible alternative completions in separate rows
        print("Start unnesting...")
        updated = unnest_dataset_lists(
            ds_updated.select_columns(
                ["prompt", "new_completions", "new_labels", "correctness"]
            ),
            ["new_completions", "new_labels"]
        ).rename_columns(
            {"new_completions": "completions", "new_labels": "labels"}
        ).select_columns(["prompt", "completions", "labels", "index", "correctness"])  # Sort the column names

        datasets.append(updated)
    
    full_ds[split] = concatenate_datasets(datasets)

Split: train
phase: 1


100%|██████████| 949/949 [00:00<00:00, 14964.00it/s]


Map (num_proc=8):   0%|          | 0/949 [00:00<?, ? examples/s]

Start unnesting...
phase: 2


100%|██████████| 97782/97782 [00:05<00:00, 17410.19it/s]


...

Map (num_proc=8):   0%|          | 0/106 [00:00<?, ? examples/s]

Start unnesting...
phase: 2


100%|██████████| 2762/2762 [00:00<00:00, 19535.43it/s]


Map (num_proc=8):   0%|          | 0/2762 [00:00<?, ? examples/s]

Start unnesting...


In [23]:
df = full_ds['test'].to_pandas()
df[:10]

,prompt,completions,labels,index,correctness
0,Three pencils and a jumbo eraser cost $\$1.24$...,[Let's call the price of a pencil p and the pr...,"[True, True, True, True, True, False]",0,solution
1,Three pencils and a jumbo eraser cost $\$1.24$...,[Let's call the price of a pencil p and the pr...,"[True, True, True, True, True, False]",0,solution
2,Three pencils and a jumbo eraser cost $\$1.24$...,[Let's call the price of a pencil p and the pr...,"[True, True, True, True, True, False]",0,solution
3,Three pencils and a jumbo eraser cost $\$1.24$...,[Let's call the price of a pencil p and the pr...,"[True, True, True, True, True, False]",0,solution
4,Three pencils and a jumbo eraser cost $\$1.24$...,[Let's call the price of a pencil p and the pr...,"[True, True, True, True, True, False]",0,solution
5,"Steve says to Jon, ""I am thinking of a polynom...","[Hey, you know Vieta's formulas, right?, Ok, s...","[True, True, True, True, True, True, True, Tru...",1,give_up
6,"Steve says to Jon, ""I am thinking of a polynom...","[Hey, you know Vieta's formulas, right?, Ok, s...","[True, True, True, True, True, True, True, Tru...",1,give_up
7,"Steve says to Jon, ""I am thinking of a polynom...","[Hey, you know Vieta's formulas, right?, Ok, s...","[True, True, True, True, True, True, True, Tru...",1,give_up
8,"Steve says to Jon, ""I am thinking of a polynom...","[Hey, you know Vieta's formulas, right?, Ok, s...","[True, True, True, True, True, True, True, Tru...",1,give_up
9,"Steve says to Jon, ""I am thinking of a polynom...","[Hey, you know Vieta's formulas, right?, Ok, s...","[True, True, True, True, True, True, True, Tru...",1,give_up


In [24]:
exhaustive=False

from datasets import Dataset
dataset = DatasetDict()
for split_type in ['test', 'train']:
    df = full_ds[split_type].to_pandas()
    train_split = pd.read_json(path_or_buf=prm_split_path[split_type], lines=True)
    df = df.merge(train_split, left_on='prompt', right_on='problem', how='inner')
    df = df.drop(columns=['solution', 'problem'])
    
    if exhaustive:
        rows = []
        for idx, row in tqdm(df.iterrows(), total=len(df)):
            for i in range(len(row["completions"])):
                new_row = {}
                new_row["prompt"] = row["prompt"]
                new_row["answer"] = row["answer"]
                new_row["subject"] = row["subject"]
                new_row["level"] = row["level"]
                new_row["completions"] = row["completions"][:i+1]
                new_row["labels"] = row["labels"][:i+1]
                new_row["index"] = row["index"]
                rows.append(new_row)
    
        df = pd.DataFrame(rows)
    
    
    
    df['completions_str'] = df['completions'].apply(lambda x: str(x))
    df['labels_str'] = df['labels'].apply(lambda x: str(x))
    df['prompt_str'] = df['prompt'].astype(str)
    df['unique_id'] = df['completions_str'] + df['prompt_str'] + df['labels_str']
    print(f"Total unique completions in train set: {len(df['unique_id'].unique())} out of {len(df)} total completions.")

    ## only keep unique rows
    df = df.drop_duplicates(subset=['unique_id'])
    df = df.drop(columns=['completions_str', 'labels_str', 'prompt_str', 'unique_id'])

    print("Length of dataset after removing duplicates:", len(df))
    dataset[split_type] = Dataset.from_pandas(df)

Total unique completions in train set: 9956 out of 10246 total completions.
Length of dataset after removing duplicates: 9956
Total unique completions in train set: 327698 out of 377938 total completions.
Length of dataset after removing duplicates: 327698


In [29]:
new_dataset = {
}
from datasets import Dataset, DatasetDict
for split in dataset:
    df = dataset[split].to_pandas()
    df.drop(columns=['__index_level_0__'], inplace=True)
    df = df[df['correctness']!='give_up']
    df.loc[df['correctness'] == 'solution', 'correctness'] = 1
    df.loc[df['correctness'] == 'found_error', 'correctness'] = 0
    new_dataset[split] = Dataset.from_pandas(df)
    display(df)


,prompt,completions,labels,index,correctness,answer,subject,level
0,Three pencils and a jumbo eraser cost $\$1.24$...,[Let's call the price of a pencil p and the pr...,"[True, True, True, True, True, False]",0,1,29,Algebra,2
1,Three pencils and a jumbo eraser cost $\$1.24$...,[Let's call the price of a pencil p and the pr...,"[True, True, True, True, True, False]",0,1,29,Algebra,2
2,Three pencils and a jumbo eraser cost $\$1.24$...,[Let's call the price of a pencil p and the pr...,"[True, True, True, True, True, False]",0,1,29,Algebra,2
3,Three pencils and a jumbo eraser cost $\$1.24$...,[Let's call the price of a pencil p and the pr...,"[True, True, True, True, True, False]",0,1,29,Algebra,2
4,Three pencils and a jumbo eraser cost $\$1.24$...,[Let's call the price of a pencil p and the pr...,"[True, True, True, True, True, False]",0,1,29,Algebra,2
...,...,...,...,...,...,...,...,...
9951,"The point $(0,0)$ is reflected over the vertic...",[I need to find the coordinates of the final p...,"[True, False]",2710,0,"(2,4)",Geometry,2
9952,"The point $(0,0)$ is reflected over the vertic...",[I need to find the coordinates of the final p...,"[True, True]",2710,0,"(2,4)",Geometry,2
9953,"The point $(0,0)$ is reflected over the vertic...",[I need to find the coordinates of the final p...,"[True, False]",2710,0,"(2,4)",Geometry,2
9954,"The point $(0,0)$ is reflected over the vertic...",[I need to find the coordinates of the final p...,"[True, False]",2710,0,"(2,4)",Geometry,2


,prompt,completions,labels,index,correctness,answer,subject,level
0,How many positive two-digit integers leave a r...,[So if a number leaves a remainder of 2 when d...,"[True, True, True, True, True, True, True, Tru...",1,1,12,Number Theory,2
1,How many positive two-digit integers leave a r...,[So if a number leaves a remainder of 2 when d...,"[True, True, True, True, True, True, True, Tru...",1,1,12,Number Theory,2
2,How many positive two-digit integers leave a r...,[So if a number leaves a remainder of 2 when d...,"[True, True, True, True, True, True, True, Tru...",1,1,12,Number Theory,2
3,How many positive two-digit integers leave a r...,[So if a number leaves a remainder of 2 when d...,"[True, True, True, True, True, True, True, Tru...",1,1,12,Number Theory,2
4,How many positive two-digit integers leave a r...,[So if a number leaves a remainder of 2 when d...,"[True, True, True, True, True, True, True, Tru...",1,1,12,Number Theory,2
...,...,...,...,...,...,...,...,...
327693,Find the greatest common divisor of 12 and 20.,"[To find the greatest common divisor, we need ...",[False],97732,0,4,Prealgebra,2
327694,Find the greatest common divisor of 12 and 20.,[Let's write out the prime factorization of 12.],[True],97732,0,4,Prealgebra,2
327695,Find the greatest common divisor of 12 and 20.,[Let's write the prime factorization for each ...,[True],97732,0,4,Prealgebra,2
327696,Find the greatest common divisor of 12 and 20.,[Let's prime factorize 12. 12 is divisible by ...,[True],97732,0,4,Prealgebra,2


In [ ]:
new_dataset = DatasetDict(new_dataset)
new_dataset.save_to_disk("../PRM800k_cleaned")

new_dataset.push_to_hub("FrozenWolf/prm800k", token="")

Saving the dataset (0/1 shards):   0%|          | 0/9881 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/326767 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/327 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/683 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/FrozenWolf/prm800k/commit/9548c90b4de1eb9694e4c488fccaf7f917953a6a', commit_message='Upload dataset', commit_description='', oid='9548c90b4de1eb9694e4c488fccaf7f917953a6a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/FrozenWolf/prm800k', endpoint='https://huggingface.co', repo_type='dataset', repo_id='FrozenWolf/prm800k'), pr_revision=None, pr_num=None)

In [ ]:
dataset = new_dataset
dataset['train'], dataset['test']

(Dataset({
     features: ['prompt', 'completions', 'labels', 'index', 'correctness', 'answer', 'subject', 'level', '__index_level_0__'],
     num_rows: 327698
 }),
 Dataset({
     features: ['prompt', 'completions', 'labels', 'index', 'correctness', 'answer', 'subject', 'level', '__index_level_0__'],
     num_rows: 9956
 }))

In [15]:
from datasets import load_dataset
dataset = load_dataset("FrozenWolf/prm800k")


In [4]:
dataset.save_to_disk("../PRM800k_cleaned")

Saving the dataset (0/1 shards):   0%|          | 0/9956 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/327698 [00:00<?, ? examples/s]

## GEMINI AS META DATA

In [38]:

path = "AIME_Dataset_1983_2024_with_gemini_responses.csv"

import pandas as pd
output = pd.read_csv(path)
output


,ID,Year,Problem Number,Question,Answer,Part,generated_completion,cleaned_generated_completion,correctness
0,1983-1,1983,1,"Let $x$ , $y$ and $z$ all exceed $1$ and let $...",60,NaN,Here is a step-by-step reasoning to solve the ...,Here is a step-by-step reasoning to solve the ...,True
1,1983-2,1983,2,"Let $f(x)=|x-p|+|x-15|+|x-p-15|$ , where $0 < ...",15,NaN,Here is a step-by-step reasoning to determine ...,Here is a step-by-step reasoning to determine ...,True
2,1983-3,1983,3,What is the product of the real roots of the e...,20,NaN,Here is a step-by-step reasoning to solve the ...,Here is a step-by-step reasoning to solve the ...,True
3,1983-4,1983,4,A machine-shop cutting tool has the shape of a...,26,NaN,Here is a step-by-step reasoning to solve the ...,Here is a step-by-step reasoning to solve the ...,True
4,1983-5,1983,5,Suppose that the sum of the squares of two com...,4,NaN,Let $x$ and $y$ be two complex numbers. We are...,Let $x$ and $y$ be two complex numbers. We are...,True
...,...,...,...,...,...,...,...,...,...
928,2024-II-11,2024,11,Find the number of triples of nonnegative inte...,601,II,Here is a step-by-step derivation to find the ...,Here is a step-by-step derivation to find the ...,True
929,2024-II-12,2024,12,"Let $O(0,0),A(\tfrac{1}{2},0),$ and $B(0,\tfra...",23,II,"Let the points be $O(0,0)$, $A(\frac{1}{2}, 0)...","Let the points be $O(0,0)$, $A(\frac{1}{2}, 0)...",True
930,2024-II-13,2024,13,Let $\omega\neq 1$ be a 13th root of unity. Fi...,321,II,Here is a step-by-step reasoning to find the r...,Here is a step-by-step reasoning to find the r...,True
931,2024-II-14,2024,14,Let $b \geq 2$ be an integer. Call a positive ...,211,II,Let $n$ be a $b$-eautiful integer. By definiti...,Let $n$ be a $b$-eautiful integer. By definiti...,True


In [40]:
# Question: 'prompt', cleaned_generated_completion: 'completions', correctness: 'labels', native 'index', correctness: 'correctness', Answer:'answer', 

output = output.rename(columns={
    'Question': 'prompt',
    'cleaned_generated_completion': 'completions',
    'index': 'index',
    'Answer': 'answer'
})

output['completions'] = output['completions'].apply(lambda x: x.split('\n\n'))
labels = []
#### ORM assumption:
for correctness, completions in zip(output['correctness'], output['completions']):
    labels.append([correctness]*len(completions))
    
output['labels'] = labels

output

,ID,Year,Problem Number,prompt,answer,Part,generated_completion,completions,correctness,labels
0,1983-1,1983,1,"Let $x$ , $y$ and $z$ all exceed $1$ and let $...",60,NaN,Here is a step-by-step reasoning to solve the ...,[Here is a step-by-step reasoning to solve the...,True,"[True, True, True, True, True, True, True, Tru..."
1,1983-2,1983,2,"Let $f(x)=|x-p|+|x-15|+|x-p-15|$ , where $0 < ...",15,NaN,Here is a step-by-step reasoning to determine ...,[Here is a step-by-step reasoning to determine...,True,"[True, True, True, True, True, True, True, Tru..."
2,1983-3,1983,3,What is the product of the real roots of the e...,20,NaN,Here is a step-by-step reasoning to solve the ...,[Here is a step-by-step reasoning to solve the...,True,"[True, True, True, True, True, True, True, Tru..."
3,1983-4,1983,4,A machine-shop cutting tool has the shape of a...,26,NaN,Here is a step-by-step reasoning to solve the ...,[Here is a step-by-step reasoning to solve the...,True,"[True, True, True, True, True, True, True, Tru..."
4,1983-5,1983,5,Suppose that the sum of the squares of two com...,4,NaN,Let $x$ and $y$ be two complex numbers. We are...,[Let $x$ and $y$ be two complex numbers. We ar...,True,"[True, True, True, True, True, True, True, Tru..."
...,...,...,...,...,...,...,...,...,...,...
928,2024-II-11,2024,11,Find the number of triples of nonnegative inte...,601,II,Here is a step-by-step derivation to find the ...,[Here is a step-by-step derivation to find the...,True,"[True, True, True, True, True, True, True, Tru..."
929,2024-II-12,2024,12,"Let $O(0,0),A(\tfrac{1}{2},0),$ and $B(0,\tfra...",23,II,"Let the points be $O(0,0)$, $A(\frac{1}{2}, 0)...","[Let the points be $O(0,0)$, $A(\frac{1}{2}, 0...",True,"[True, True, True, True, True, True, True, Tru..."
930,2024-II-13,2024,13,Let $\omega\neq 1$ be a 13th root of unity. Fi...,321,II,Here is a step-by-step reasoning to find the r...,[Here is a step-by-step reasoning to find the ...,True,"[True, True, True, True, True, True, True, Tru..."
931,2024-II-14,2024,14,Let $b \geq 2$ be an integer. Call a positive ...,211,II,Let $n$ be a $b$-eautiful integer. By definiti...,[Let $n$ be a $b$-eautiful integer. By definit...,True,"[True, True, True, True, True, True, True, Tru..."


In [42]:
output['index'] = output.index
output

,ID,Year,Problem Number,prompt,answer,Part,generated_completion,completions,correctness,labels,index
0,1983-1,1983,1,"Let $x$ , $y$ and $z$ all exceed $1$ and let $...",60,NaN,Here is a step-by-step reasoning to solve the ...,[Here is a step-by-step reasoning to solve the...,True,"[True, True, True, True, True, True, True, Tru...",0
1,1983-2,1983,2,"Let $f(x)=|x-p|+|x-15|+|x-p-15|$ , where $0 < ...",15,NaN,Here is a step-by-step reasoning to determine ...,[Here is a step-by-step reasoning to determine...,True,"[True, True, True, True, True, True, True, Tru...",1
2,1983-3,1983,3,What is the product of the real roots of the e...,20,NaN,Here is a step-by-step reasoning to solve the ...,[Here is a step-by-step reasoning to solve the...,True,"[True, True, True, True, True, True, True, Tru...",2
3,1983-4,1983,4,A machine-shop cutting tool has the shape of a...,26,NaN,Here is a step-by-step reasoning to solve the ...,[Here is a step-by-step reasoning to solve the...,True,"[True, True, True, True, True, True, True, Tru...",3
4,1983-5,1983,5,Suppose that the sum of the squares of two com...,4,NaN,Let $x$ and $y$ be two complex numbers. We are...,[Let $x$ and $y$ be two complex numbers. We ar...,True,"[True, True, True, True, True, True, True, Tru...",4
...,...,...,...,...,...,...,...,...,...,...,...
928,2024-II-11,2024,11,Find the number of triples of nonnegative inte...,601,II,Here is a step-by-step derivation to find the ...,[Here is a step-by-step derivation to find the...,True,"[True, True, True, True, True, True, True, Tru...",928
929,2024-II-12,2024,12,"Let $O(0,0),A(\tfrac{1}{2},0),$ and $B(0,\tfra...",23,II,"Let the points be $O(0,0)$, $A(\frac{1}{2}, 0)...","[Let the points be $O(0,0)$, $A(\frac{1}{2}, 0...",True,"[True, True, True, True, True, True, True, Tru...",929
930,2024-II-13,2024,13,Let $\omega\neq 1$ be a 13th root of unity. Fi...,321,II,Here is a step-by-step reasoning to find the r...,[Here is a step-by-step reasoning to find the ...,True,"[True, True, True, True, True, True, True, Tru...",930
931,2024-II-14,2024,14,Let $b \geq 2$ be an integer. Call a positive ...,211,II,Let $n$ be a $b$-eautiful integer. By definiti...,[Let $n$ be a $b$-eautiful integer. By definit...,True,"[True, True, True, True, True, True, True, Tru...",931


In [44]:
from datasets import Dataset
dataset = Dataset.from_pandas(output)
dataset

Dataset({
    features: ['ID', 'Year', 'Problem Number', 'prompt', 'answer', 'Part', 'generated_completion', 'completions', 'correctness', 'labels', 'index'],
    num_rows: 933
})

In [ ]:
dataset.save_to_disk('../Gemini-AIME-Meta')
dataset.push_to_hub("FrozenWolf/Gemini-AIME-Meta", token="")

Saving the dataset (0/1 shards):   0%|          | 0/933 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/FrozenWolf/Gemini-AIME-Meta/commit/03dbdb9942a45548ba18b286bf1b5466303d8360', commit_message='Upload dataset', commit_description='', oid='03dbdb9942a45548ba18b286bf1b5466303d8360', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/FrozenWolf/Gemini-AIME-Meta', endpoint='https://huggingface.co', repo_type='dataset', repo_id='FrozenWolf/Gemini-AIME-Meta'), pr_revision=None, pr_num=None)

In [248]:
dataset

DatasetDict({
    test: Dataset({
        features: ['prompt', 'completions', 'labels', 'index', 'answer', 'subject', 'level'],
        num_rows: 9956
    })
    train: Dataset({
        features: ['prompt', 'completions', 'labels', 'index', 'answer', 'subject', 'level'],
        num_rows: 327797
    })
})

### DATALOADER TESTING

In [1]:
import torch
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import re
from datasets import load_dataset, Dataset
import pandas as pd
import os
import numpy as np
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Math-1.5B-Instruct")
tokenizer.add_tokens(["<PRM_STEP_SCORE>"])
data = load_dataset("FrozenWolf/prm800k")

subjects_map = {'Algebra':0,
 'Counting & Probability':1,
 'Geometry':2,
 'Intermediate Algebra':3,
 'Number Theory':4,
 'Prealgebra':5,
 'Precalculus':6,
 'Others':7}


def chat_template( question, steps):
        steps = ' <PRM_STEP_SCORE> \n'.join(steps)
        
        prompt = f'''Question:
{question}
Answer:
{steps} <PRM_STEP_SCORE>'''
        
        return prompt
    
def chat_template_no_special( question, steps):
        steps = '\n'.join(steps)
        
        prompt = f'''Question:
{question}
Answer:
{steps} <PRM_STEP_SCORE>'''
        
        return prompt





class QwenMathDataset(Dataset):
    '''
    This one generates a single trajectory per iteration and ALL steps accuracy like:
    [A, B, C] -> [T, F, T]
    '''
    def __init__(self, data, tokenizer, special_tokens=True, inference=False):
        self.dataset = data
        self.tokenizer = tokenizer
        self.special_tokens = special_tokens
        self.inference = inference
        
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        print("index", idx, type(idx))
        prompt = self.dataset[idx]['prompt']
        completions = self.dataset[idx]['completions']
        raw_labels = self.dataset[idx]['labels']
        print("raw_labels", raw_labels, idx, subjects_map)
        dset = subjects_map[self.dataset[idx]['subject']]

        if self.special_tokens:
            text = chat_template(prompt, completions)
        else:
            text = chat_template_no_special(prompt, completions)    
        

        model_inputs = self.tokenizer([text], return_tensors="pt")
        labels = torch.ones_like(model_inputs['input_ids']).float()

        if self.special_tokens:
            labels[(model_inputs['input_ids']!=len(self.tokenizer)-1)] = -100
            labels[(model_inputs['input_ids']==len(self.tokenizer)-1)] = torch.tensor(raw_labels).float()
        else:
            labels *= -100
            labels[-1] = torch.tensor(raw_labels).float()[-1]

        model_inputs["dataset"] = dset
        model_inputs["labels"] = labels
        

        return [model_inputs['input_ids']], [model_inputs['attention_mask']], [labels], [dset]
    



qwen_ds = QwenMathDataset(data['train'], tokenizer, special_tokens=True)
next(iter(qwen_ds))

AttributeError: 'QwenMathDataset' object has no attribute '_indices'

In [1]:
from prm_data import *
## reload this dataset class
# import importlib
# importlib.reload(prm_data)

from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForTokenClassification

model_name = "Qwen/Qwen2-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_tokens(['<PRM_STEP_SCORE>'])

    
(
    train_dataloader,
    meta_dataloader,
    test_dataloader,
) = build_dataloader(
    tokenizer=tokenizer,
    train_batch_size= 2,
    meta_batch_size= 1,
    token_based=True,
    meta_dataset="AIME",
)



batch = next(iter(meta_dataloader))
batch['index']

Qwen2TokenizerFast(name_or_path='Qwen/Qwen2-0.5B', vocab_size=151643, model_max_length=32768, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<PRM_STEP_SCORE>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
}
) 2 1 True AIME False
Loading aime_outputs/test2025-II/o4-mini/AIME_o4_mini_8_traj_25_II.jsonl with 15 samples
[0]
Loading aime_outputs/test2025-II/qwen-instruct-1.5B/test2025-II_

[509]

In [13]:
import pandas as pd
df = pd.read_json(path_or_buf="aime_outputs/test2025-I/o4-mini/AIME_o4_mini_8_traj_25_I.jsonl", lines=True)
df

,id,problem,generated_responses,generated_answers,gold_answer,is_correct,answers_correctness,majority_voting,majority_voting_is_correct,prompt
0,I-1,Find the sum of all integer bases $b > 9$ for ...,[We work in an integer base \(b>9\). In base ...,"[70, 70, 70, 70, 70, 70, 70, 70]",70,True,"[True, True, True, True, True, True, True, True]",70,True,Find the sum of all integer bases $b > 9$ for ...
1,I-2,"On $\triangle ABC$, points $A$, $D$, $E$, and ...",[**Solution.** \nWe begin by introducing conv...,"[588, 588, 588, 588, 588, 588, 588, 588]",588,True,"[True, True, True, True, True, True, True, True]",588,True,"On $\triangle ABC$, points $A$, $D$, $E$, and ..."
2,I-3,The 9 members of a baseball team went to an ic...,"[Solution.\n\nLabel the nine players as 1,2,…,...","[16, 16, 16, 16, 16, 16, 16, 16]",16,True,"[True, True, True, True, True, True, True, True]",16,True,The 9 members of a baseball team went to an ic...
3,I-4,"Find the number of ordered pairs $(x,y)$, wher...","[We wish to count integer solutions \((x,y)\in...","[117, 117, 117, 117, 117, 117, 117, 117]",117,True,"[True, True, True, True, True, True, True, True]",117,True,"Find the number of ordered pairs $(x,y)$, wher..."
4,I-5,There are $8! = 40320$ eight-digit positive in...,"[We must count the 8‐digit permutations of 1,2...","[279, 279, 279, 279, 279, 279, 279, 279]",279,True,"[True, True, True, True, True, True, True, True]",279,True,There are $8! = 40320$ eight-digit positive in...
5,I-6,An isosceles trapezoid has an inscribed circle...,[Here is a complete solution.\n\n**Solution.**...,"[504, 504, 504, 504, 504, 504, 504, 504]",504,True,"[True, True, True, True, True, True, True, True]",504,True,An isosceles trapezoid has an inscribed circle...
6,I-7,"The twelve letters $A$, $B$, $C$, $D$, $E$, $F...",[First we label the twelve letters by their us...,"[821, 79, 821, 821, 821, 710, 821, 821]",821,True,"[True, False, True, True, True, False, True, T...",821,True,"The twelve letters $A$, $B$, $C$, $D$, $E$, $F..."
7,I-8,Let $k$ be a real number such that the system\...,[We identify complex numbers \(z=x+iy\) with p...,"[77, 77, 77, 77, 77, 77, 77, 77]",77,True,"[True, True, True, True, True, True, True, True]",77,True,Let $k$ be a real number such that the system\...
8,I-9,The parabola with equation $y = x^2 - 4$ is ro...,[We begin by parameterize the original parabol...,"[62, , 62, 62, 62, 62, 62, 62]",62,True,"[True, False, True, True, True, True, True, True]",62,True,The parabola with equation $y = x^2 - 4$ is ro...
9,I-10,The 27 cells of a $3 \times 9$ grid are filled...,"[We label the rows by 1,2,3, the columns by 1,...","[81, 81, 81, 81, 81, 81, 81, 119]",81,True,"[True, True, True, True, True, True, True, False]",81,True,The 27 cells of a $3 \times 9$ grid are filled...


In [18]:
for i in df['generated_responses'].iloc[1][1].split('\n\n'):
    print(i)
    print("==="*10)

**Solution.**  We begin by choosing convenient coordinates so that the computations become purely algebraic.  Let 
\[
A=(0,0),
\quad B=(28,0),
\]
so that \(AB=28=4+16+8\), and write
\[
C=(X,Y),
\]
with \(AC=91=13+52+26\) so
\[
X^2+Y^2=91^2=8281.
\]
On \(AB\) we have
\[
D\text{ at distance }4\text{ from }A\;\Longrightarrow\;D=(4,0),
\quad
E\text{ at distance }4+16=20\text{ from }A\;\Longrightarrow\;E=(20,0).
\]
On \(AC\), since \(AF=13\) and \(AC=91\), the point \(F\) is the \(\tfrac{13}{91}\)-point of \(AC\):
\[
F=\frac{13}{91}C=\Bigl(\tfrac{13}{91}X,\;\tfrac{13}{91}Y\Bigr).
\]
Similarly \(G\) lies so that \(AG=13+52=65\), hence
\[
G=\frac{65}{91}C=\Bigl(\tfrac{65}{91}X,\;\tfrac{65}{91}Y\Bigr).
\]
Next \(M\) is the reflection of \(D\) across \(F\), so
\[
M=2F-D
=\Bigl(2\cdot\tfrac{13}{91}X-4,\;2\cdot\tfrac{13}{91}Y-0\Bigr)
=\Bigl(\tfrac{26}{91}X-4,\;\tfrac{26}{91}Y\Bigr).
\]
And \(N\) is the reflection of \(G\) across \(E\), so
\[
N=2E-G
=\Bigl(2\cdot20-\tfrac{65}{91}X,\;0- \tfrac{65}{

In [3]:
batch['correctness']

[[True]]

In [2]:
batch['index']

[92, 92, 92, 92, 92]

In [20]:
model

QwenMathCondGen_RM(
  (base_model): Qwen2ForCausalLM(
    (model): Qwen2Model(
      (embed_tokens): Embedding(151647, 896)
      (layers): ModuleList(
        (0-23): 24 x Qwen2DecoderLayer(
          (self_attn): Qwen2Attention(
            (q_proj): Linear(in_features=896, out_features=896, bias=True)
            (k_proj): Linear(in_features=896, out_features=128, bias=True)
            (v_proj): Linear(in_features=896, out_features=128, bias=True)
            (o_proj): Linear(in_features=896, out_features=896, bias=False)
          )
          (mlp): Qwen2MLP(
            (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
            (up_proj): Linear(in_features=896, out_features=4864, bias=False)
            (down_proj): Linear(in_features=4864, out_features=896, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
          (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        )
      )
 

In [19]:
from model import *
model = QwenMathCondGen_RM("cuda", "Qwen/Qwen2-0.5B")
model.to("cuda")
# model = QwenMathCondGen_RM("cuda", "Qwen/Qwen2-0.5B")

/home/frozenwolf/miniconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/frozenwolf/miniconda3/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Model dtype: torch.bfloat16
Added 1 new tokens to the tokenizer and model. The token is <PRM_STEP_SCORE>.


QwenMathCondGen_RM(
  (base_model): Qwen2ForCausalLM(
    (model): Qwen2Model(
      (embed_tokens): Embedding(151647, 896)
      (layers): ModuleList(
        (0-23): 24 x Qwen2DecoderLayer(
          (self_attn): Qwen2Attention(
            (q_proj): Linear(in_features=896, out_features=896, bias=True)
            (k_proj): Linear(in_features=896, out_features=128, bias=True)
            (v_proj): Linear(in_features=896, out_features=128, bias=True)
            (o_proj): Linear(in_features=896, out_features=896, bias=False)
          )
          (mlp): Qwen2MLP(
            (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
            (up_proj): Linear(in_features=896, out_features=4864, bias=False)
            (down_proj): Linear(in_features=4864, out_features=896, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
          (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        )
      )
 

In [2]:
batch = next(iter(test_dataloader['test2025-II']['aime_outputs/test2025-II/o4-mini']))
batch['label'].shape, batch['index']

(torch.Size([65, 838]),
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0])

In [20]:
test_dataloader['test2025-I']['aime_outputs/test2025-I/o4-mini'].dataset.dataset

Dataset({
    features: ['id', 'problem', 'completions', 'generated_answers', 'gold_answer', 'is_correct', 'answers_correctness', 'majority_voting', 'majority_voting_is_correct', 'prompt', 'index', 'labels', 'subject'],
    num_rows: 120
})

In [5]:
# meta_dataloader.dataset.dataset['correctness']

In [6]:
batch = next(iter(meta_dataloader))
batch['label'].shape

torch.Size([11, 2862])

In [9]:

device = "cuda"
labels = batch['label'].to(device) ## (B, T)
correctness = torch.tensor(batch['correctness'], dtype=torch.float).to(device) ## (B, )
score = model(batch['input_ids'].to(device),
                                     batch['attention_mask'].to(device))

In [10]:
score.shape, labels.shape, correctness.shape

(torch.Size([4]), torch.Size([4, 215]), torch.Size([1, 1]))

In [11]:
batch['index']

[6219, 6219, 6219, 6219]

In [12]:
criterion = nn.MSELoss(reduction='none')
criterion_meta = nn.MSELoss()
criterion_CE = nn.BCEWithLogitsLoss(reduction='none')


nproblems = set(batch['index']) # [0, 1, 2, B_Size-1]
# score -> (B * T*(T+1)/2) So [A_0_1, A_0_2,.. B_0_1, B_0_2,...] in cummulative order
score = torch.log(score / (1 - score))
outputs = []
for i in nproblems:
    mean_score = 0
    step = 0
    for j in range(len(score)):
        if batch['index'][j] == i:
            mean_score += score[j]
            step += 1
    mean_score /= step
    
    outputs.append(mean_score)
    
## outputs -> (B, )
## label -> ## (B * T*(T+1)/2)
outputs = torch.stack(outputs) # (B, )
outputs = torch.sigmoid(outputs)
loss = criterion_meta(outputs, correctness)

/home/frozenwolf/miniconda3/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [19]:
non_filler = (labels != -100).float()
index = torch.argmax(non_filler, dim=1)
labels = labels[torch.arange(labels.size(0)),index]  # (B, )
loss = criterion(score, labels)
        

In [25]:
nproblems = set(batch['index']) # [0, 1, 2, B_Size-1]
score = torch.log(score / (1 - score))
outputs = []
for i in nproblems:
    mean_score = 0
    step = 0
    for j in range(len(score)):
        if batch['index'][j] == i:
            mean_score += score[j]
            step += 1
    mean_score /= step
    outputs.append(mean_score)
outputs = torch.stack(outputs) # (B, )
outputs = torch.sigmoid(outputs)

ZeroDivisionError: division by zero

In [26]:
batch['index']

[7024, 7024, 8032, 8032, 7133, 7133, 7133, 7133, 7133]

In [9]:
outputs

tensor([0.9453, 0.9297, 0.9219], device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<SigmoidBackward0>)

In [5]:
load_dataset("FrozenWolf/Gemini-AIME-Meta")['train']

Dataset({
    features: ['ID', 'Year', 'Problem Number', 'prompt', 'answer', 'Part', 'generated_completion', 'completions', 'correctness', 'labels', 'index'],
    num_rows: 933
})

In [1]:
import torch
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import re
from datasets import load_dataset, Dataset
import pandas as pd
import os
import numpy as np
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Math-1.5B-Instruct")
tokenizer.add_tokens(["<PRM_STEP_SCORE>"])
dataset = load_dataset("FrozenWolf/prm800k")


    
    
subjects_map = {'Algebra':0,
 'Counting & Probability':1,
 'Geometry':2,
 'Intermediate Algebra':3,
 'Number Theory':4,
 'Prealgebra':5,
 'Precalculus':6,
 'Others':7}

def chat_template( question, steps):
        steps = ' <PRM_STEP_SCORE> \n'.join(steps)
        
        prompt = f'''Question:
{question}
Answer:
{steps} <PRM_STEP_SCORE>'''
        
        return prompt
    
def chat_template_no_special( question, steps):
        steps = '\n'.join(steps)
        
        prompt = f'''Question:
{question}
Answer:
{steps} <PRM_STEP_SCORE>'''
        
        return prompt


class QwenMathDataset(Dataset):
    '''
    This one generates a single trajectory per iteration and ALL steps accuracy like:
    [A, B, C] -> [T, F, T]
    '''
    def __init__(self, data, tokenizer, special_tokens=True, inference=False):
        self.dataset = data
        self.tokenizer = tokenizer
        self.special_tokens = special_tokens
        self.inference = inference
        
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        print("index", idx, type(idx))
        prompt = self.dataset[idx]['prompt']
        completions = self.dataset[idx]['completions']
        raw_labels = self.dataset[idx]['labels']
        print("raw_labels", raw_labels, idx, subjects_map)
        dset = subjects_map[self.dataset[idx]['subject']]

        if self.special_tokens:
            text = chat_template(prompt, completions)
        else:
            text = chat_template_no_special(prompt, completions)    
        

        model_inputs = self.tokenizer([text], return_tensors="pt")
        labels = torch.ones_like(model_inputs['input_ids']).float()

        if self.special_tokens:
            labels[(model_inputs['input_ids']!=len(self.tokenizer)-1)] = -100
            labels[(model_inputs['input_ids']==len(self.tokenizer)-1)] = torch.tensor(raw_labels).float()
        else:
            labels *= -100
            labels[-1] = torch.tensor(raw_labels).float()[-1]

        model_inputs["dataset"] = dset
        model_inputs["labels"] = labels
        

        return [model_inputs['input_ids']], [model_inputs['attention_mask']], [labels], [dset]
    
    
ds = QwenMathDataset(dataset['train'], tokenizer, special_tokens=True, inference=False)
next(iter(ds))

AttributeError: 'QwenMathDataset' object has no attribute '_indices'

In [3]:
def collate_merge_minibatch(batch):
    if len(batch[0]) == 4:
        names = ["input_ids", "attention_mask", "labels", "index"]
    elif len(batch[0]) == 5:
        names = ["input_ids", "attention_mask", "labels", "dataset", "correctness"]
    else:
        raise ValueError("Batch items must contain 4 or 5 elements.")
    merged = {}
    for name_idx, item in enumerate(zip(*batch)):
        j = []
        for i in item:
            j+=i
        if isinstance(j[0], torch.Tensor):
            max_len = max([i.shape[-1] for i in j ])
        for i in range(len(j)):    
            if isinstance(j[i], torch.Tensor):
                if names[name_idx] == "input_ids":
                    j[i] = torch.cat((j[i], torch.ones(1, max_len - j[i].shape[-1])*tokenizer.pad_token_id), dim=-1)
                elif names[name_idx] == "attention_mask":
                    j[i] = torch.cat((j[i], torch.zeros(1, max_len - j[i].shape[-1])), dim=-1)
                elif names[name_idx] == "labels":
                    j[i] = torch.cat((j[i], torch.ones(1, max_len - j[i].shape[-1])*-100), dim=-1)
                elif names[name_idx] == "correctness":
                    j[i] = torch.tensor(j[i]).float()
        if isinstance(j[0], torch.Tensor):
            j = torch.cat(j, dim=0)
        merged[names[name_idx]] = j
    return merged

In [4]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(ds, batch_size=4, shuffle=True, collate_fn=collate_merge_minibatch)

next(iter(train_dataloader))  

index 67031 <class 'int'>
raw_labels [True, True, True, True, True, True, True, True, True, True, False] 67031 {'Algebra': 0, 'Counting & Probability': 1, 'Geometry': 2, 'Intermediate Algebra': 3, 'Number Theory': 4, 'Prealgebra': 5, 'Precalculus': 6, 'Others': 7}
index 21090 <class 'int'>
raw_labels [True, True, True, True, True, True, True, True, False] 21090 {'Algebra': 0, 'Counting & Probability': 1, 'Geometry': 2, 'Intermediate Algebra': 3, 'Number Theory': 4, 'Prealgebra': 5, 'Precalculus': 6, 'Others': 7}
index 169948 <class 'int'>
raw_labels [True, True] 169948 {'Algebra': 0, 'Counting & Probability': 1, 'Geometry': 2, 'Intermediate Algebra': 3, 'Number Theory': 4, 'Prealgebra': 5, 'Precalculus': 6, 'Others': 7}
index 135737 <class 'int'>
raw_labels [True, True, True] 135737 {'Algebra': 0, 'Counting & Probability': 1, 'Geometry': 2, 'Intermediate Algebra': 3, 'Number Theory': 4, 'Prealgebra': 5, 'Precalculus': 6, 'Others': 7}


{'input_ids': tensor([[ 14582.,    510.,   4340.,  ...,  12947.,    220., 151665.],
         [ 14582.,    510.,  10048.,  ..., 151643., 151643., 151643.],
         [ 14582.,    510.,  10061.,  ..., 151643., 151643., 151643.],
         [ 14582.,    510.,  10061.,  ..., 151643., 151643., 151643.]]),
 'attention_mask': tensor([[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.]]),
 'labels': tensor([[-100., -100., -100.,  ..., -100., -100.,    0.],
         [-100., -100., -100.,  ..., -100., -100., -100.],
         [-100., -100., -100.,  ..., -100., -100., -100.],
         [-100., -100., -100.,  ..., -100., -100., -100.]]),
 'index': [1, 1, 3, 6]}

In [57]:
train_dataloader.dataset.dataset[0]['subject']

'Number Theory'

In [11]:
neg = 0
pos = 0
neg_per_sent = 0
for label in dataset['train']['labels']:
    c = False
    for l in label:
        if l:
            pos += 1
        else:
            neg += 1
            if not c:
                neg_per_sent += 1
                c = True
            
print(f"Total positive labels in train set: {pos*100/(pos+neg)}, negative labels: {neg*100/(pos+neg)}")
print(f"Atleast 1 Neg labels per sentence: {neg_per_sent*100 / len(dataset['train'])}")

Total positive labels in train set: 89.96407002368821, negative labels: 10.035929976311786
Atleast 1 Neg labels per sentence: 60.82239922879099
